# Implementation of SimGAE

### 1. Firslty install several relevant libraries.

In [1]:

import torch
torchversion = torch.version.__version__
!pip install scipy
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install torch_geometric
import numpy as np
import networkx as nx
from scipy.spatial import Delaunay
import os
import torch.nn as nn
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data
import torch_geometric.transforms as T
import sys
import scipy.sparse as sp
import math
from numpy.linalg import inv, pinv
from torch.nn import Softmax
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score,average_precision_score
from torch.nn.init import xavier_normal_ as xavier



Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple


### 2. Then, define some functions for computing hodge laplacians for 2-simplex.

The version of higher simplexes is in working. 

In [2]:

def get_faces(G):
    """
    Returns a list of the faces of 2-simplexes in an undirected graph
    i.e, triangle is a 2-simplex in an undirected graph and each edge of this triangle is a face of this triangle
    """
    edges = list(G.edges)
    faces = []
    for i in range(len(edges)):
        for j in range(i+1, len(edges)):
            e1 = edges[i]
            e2 = edges[j]
            if e1[0] == e2[0]:
                shared = e1[0]
                e3 = (e1[1], e2[1])
            elif e1[1] == e2[0]:
                shared = e1[1]
                e3 = (e1[0], e2[1])
            elif e1[0] == e2[1]:
                shared = e1[0]
                e3 = (e1[1], e2[0])
            elif e1[1] == e2[1]:
                shared = e1[1]
                e3 = (e1[0], e2[0])
            else:  # edges don't connect
                continue

            if e3[0] in G[e3[1]]: # if 3rd edge is in graph
                faces.append(tuple(sorted((shared, *e3))))
    return list(sorted(set(faces)))


def incidence_matrices(G, V, E, faces, edge_to_idx):
    """
    Returns incidence matrices B1 and B2
    :param G: NetworkX DiGraph
    :param V: list of nodes
    :param E: list of edges
    :param faces: list of faces in G
    Returns B1 (|V| x |E|) and B2 (|E| x |faces|)
    B1[i][j]: -1 if node is is tail of edge j, 1 if node is head of edge j, else 0 (tail -> head) (smaller -> larger)
    B2[i][j]: 1 if edge i appears sorted in face j, -1 if edge i appears reversed in face j, else 0; given faces with sorted node order
    """
    B1 = np.array(nx.incidence_matrix(G, nodelist=V, edgelist=E, oriented=True).todense())
    B2 = np.zeros([len(E),len(faces)])

    for f_idx, face in enumerate(faces): # face is sorted
        edges = [face[:-1], face[1:], [face[0], face[2]]]
        e_idxs = [edge_to_idx[tuple(e)] for e in edges]

        B2[e_idxs[:-1], f_idx] = 1
        B2[e_idxs[-1], f_idx] = -1
    return B1, B2


def compute_D2(B):
    """
    Computes D2 = max(diag(dot(|B|, 1)), I)
    """
    B_rowsum = np.abs(B).sum(axis=1)

    D2 = np.diag(np.maximum(B_rowsum, 1))
    return D2

def compute_D1(B1, D2):
    """
    Computes D1 = 2 * max(diag(|B1|) .* D2
    """
    rowsum = (np.abs(B1) @ D2).sum(axis=1)
    D1 = 2 * np.diag(rowsum)

    return D1



def compute_hodge_matrices(B1, B2):
    """
    Computes normalized Hodge Laplacians L0 and L1 matrices,
    """
    # D matrices
    D2_2 = compute_D2(B2)
    D2_1 = compute_D2(B1)
    D3_n = np.identity(B1.shape[1]) # (|E| x |E|)
    D1 = compute_D1(B1, D2_2)
    D3 = np.identity(B2.shape[1]) / 3 # (|F| x |F|)

    # L matrices
    D1_pinv = pinv(D1)
    D2_2_inv = inv(D2_2)


    L0u = B1 @ D3_n @ B1.T @ inv(D2_1) #B1.T @ B1 
    L1u = D2_2 @ B1.T @ D1_pinv @ B1
    L1d = B2 @ D3 @ B2.T @ D2_2_inv
    L1f = L1u + L1d
    
    # print(B1.shape)
    # print(B2.shape)
    # print(L0u.shape)
    # print(L1u.shape)    
    # print(L1d.shape)
    # # print(L1f.shape)
    return L0u, L1f

def compute_boundary_matrix(data, sample_data_edge_index):
    "compute boundary matrix of Node"
    g = nx.Graph()
    g.add_nodes_from([i for i in range(len(data.y))])
    edge_index_ = np.array((sample_data_edge_index))
    edge_index = [(edge_index_[0, i], edge_index_[1, i]) for i in
                        range(np.shape(edge_index_)[1])]
    g.add_edges_from(edge_index)

    edge_to_idx = {edge: i for i, edge in enumerate(g.edges)}

    B1, B2 = incidence_matrices(g, sorted(g.nodes), sorted(g.edges), get_faces(g), edge_to_idx)
    print(B1.shape)
    print(B2.shape)

    return B1, B2


### 3. Compared to other typical GNN network, it takes hodge_laplacians of 1-simplex and 2=simplex as input. The version of higher simplexes is in working. 

In [3]:
class SIMGAE(torch.nn.Module):
    def __init__(self,data,num_features,num_classes,hodge_laplacians,dimension=8):
        super(SIMGAE, self).__init__()
        self.conv1 = GCNConv(num_features, 64, cached=True)
        self.conv2 = GCNConv(64, 32, cached=True)
        self.hodge_laplacians = hodge_laplacians
        self.leakyrelu = torch.nn.LeakyReLU(0.2, True)
        self.linear = torch.nn.Linear(32, 1, bias=True)
        self.linear_1 = torch.nn.Linear(dimension + 32, 32, bias=True)
        self.softmax = Softmax(dim=1)
        hodge_laplacian0_size = self.hodge_laplacians[0].size(0)
        hodge_laplacian1_size = self.hodge_laplacians[1].size(0)
        self.weights_sim = nn.Parameter(torch.FloatTensor(int(hodge_laplacian0_size+hodge_laplacian1_size), dimension))
        self.embeddings_sim = nn.Parameter(torch.FloatTensor(data.x.size(1), int(hodge_laplacian0_size+hodge_laplacian1_size)))

        # reset parameters
        nn.init.kaiming_uniform_(self.weights_sim, mode = 'fan_out', a = math.sqrt(5))
        nn.init.kaiming_uniform_(self.embeddings_sim, mode='fan_out', a=math.sqrt(5))

    def g_encode(self,data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x,p=0.5,training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        return x

    def s_encode(self,data,g_emb,type="train"):
        if type == 'train':
            edges_pos = data.total_edges[:data.train_pos]
            index = np.random.randint(0, data.train_neg, data.train_pos)
            edges_neg = data.total_edges[data.train_pos:data.train_pos + data.train_neg][index]
            total_edges = np.concatenate((edges_pos,edges_neg))
            edges_y = torch.cat((data.total_edges_y[:data.train_pos],data.total_edges_y[data.train_pos:data.train_pos + data.train_neg][index]))

        elif type == 'val':
            total_edges =  data.total_edges[data.train_pos+data.train_neg:data.train_pos+data.train_neg+data.val_pos+data.val_neg]
            edges_y = data.total_edges_y[data.train_pos+data.train_neg:data.train_pos+data.train_neg+data.val_pos+data.val_neg]

        elif type == 'test':
            total_edges = data.total_edges[
                          data.train_pos + data.train_neg + data.val_pos + data.val_neg:]
            edges_y = data.total_edges_y[
                      data.train_pos + data.train_neg + data.val_pos + data.val_neg :]


        L0, L1 = self.hodge_laplacians
        L0_r = torch.matrix_power(L0, 1)
        L1_r = torch.matrix_power(L1, 1)
        #print(L0.size())
        #print(L1.size())
        diag_zeros = torch.zeros(L0.size()[0], L1.size()[1]).to(L0.device)
        upper_block = torch.cat([L0_r, diag_zeros], dim=1)
        lower_block = torch.cat([diag_zeros.T, L1_r], dim=1)
        sim_block = torch.cat([upper_block, lower_block], dim=0)

        x = data.x
        embeddings_sim = torch.matmul(x, self.embeddings_sim)
        s_emb_sim_ = torch.matmul(embeddings_sim, sim_block)  
        s_emb_sim = torch.matmul(s_emb_sim_, self.weights_sim)
        s_emb_sim = s_emb_sim.renorm_(2, 0, 1)

        s_emb_sim_in = s_emb_sim[total_edges[:, 0]]
        s_emb_sim_out = s_emb_sim[total_edges[:, 1]]

        d_sim = (s_emb_sim_in - s_emb_sim_out).pow(2)

        # linear to gather edge features
        # embedding from GCN
        g_emb = g_emb.renorm_(2,0,1)
        alpha = 1.0
        beta = 0.1

        g_emb_in = g_emb[total_edges[:, 0]]
        g_emb_out = g_emb[total_edges[:, 1]]
        g_sqdist = (g_emb_in - g_emb_out).pow(2)
        sqdist = self.leakyrelu(self.linear_1(torch.cat((alpha * g_sqdist, beta * d_sim), dim=1)))
        sqdist = torch.abs(self.linear(sqdist)).reshape(-1)
        sqdist = torch.clamp(sqdist, min=0, max=40)
        prob = 1. / (torch.exp((sqdist - 2.0) / 1.0) + 1.0)
        return prob, edges_y.float()

### 4. Following the pytorch official tutorial of Link Prediction using Graph Neural Networks. Then, define some functions.

Formulate the link prediction problem as a binary classification. 

These functions are for 

1. Treat the edges in the graph as positive examples.
2. Sample a number of non-existent edges (i.e. node pairs with no edges between them) as negative examples.
Divide the positive examples and negative examples into a training set and a test set.

In [4]:
def get_edges_split(data, val_prop = 0.2, test_prop = 0.2):
    g = nx.Graph()
    g.add_nodes_from([i for i in range(len(data.y))])
    _edge_index_ = np.array((data.edge_index))
    edge_index_ = [(_edge_index_[0, i], _edge_index_[1, i]) for i in
                        range(np.shape(_edge_index_)[1])]
    g.add_edges_from(edge_index_)
    adj = nx.adjacency_matrix(g)

    return get_adj_split(adj,val_prop = val_prop, test_prop = test_prop)

def get_adj_split(adj, val_prop=0.05, test_prop=0.1):
    x, y = sp.triu(adj).nonzero()
    pos_edges = np.array(list(zip(x, y)))
    np.random.shuffle(pos_edges)
    # get tn edges
    x, y = sp.triu(sp.csr_matrix(1. - adj.toarray())).nonzero()
    neg_edges = np.array(list(zip(x, y)))
    np.random.shuffle(neg_edges)

    m_pos = len(pos_edges)
    n_val = int(m_pos * val_prop)
    n_test = int(m_pos * test_prop)
    val_edges, test_edges, train_edges = pos_edges[:n_val], pos_edges[n_val:n_test + n_val], pos_edges[n_test + n_val:]
    val_edges_false, test_edges_false = neg_edges[:n_val], neg_edges[n_val:n_test + n_val]
    train_edges_false = np.concatenate([neg_edges, val_edges, test_edges], axis=0)
    return train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false




### 5. Train, test, weights_init, setup_seed are some network training preliminary function.
### Call function interaget the loading data, creating model and setiing parameters.

In [5]:
def train():
    model.train()
    optimizer.zero_grad()
    emb = model.g_encode(data).clone()
    x, y = model.s_encode(data, emb)  # emb from encode's, i.e., Gconv's output
    loss = F.binary_cross_entropy(x,y)
    loss.backward()
    optimizer.step()
    #scheduler.step()
    return x

def test():
    model.eval()
    accs = []
    emb = model.g_encode(data)
    for type in ["val", "test"]:
        pred,y = model.s_encode(data,emb,type=type)
        pred,y = pred.cpu(),y.cpu()
        if type == "val":
            accs.append(F.binary_cross_entropy(pred, y))
            pred = pred.data.numpy()
            roc = roc_auc_score(y, pred)
            accs.append(roc)
            acc = average_precision_score(y,pred)
            accs.append(acc)
        else:
            pred = pred.data.numpy()
            roc = roc_auc_score(y, pred)
            accs.append(roc)
            acc = average_precision_score(y, pred)
            accs.append(acc)
    return accs

def weights_init(m):
    if isinstance(m, torch.nn.Linear):
        xavier(m.weight)
        if not m.bias is None:
            torch.nn.init.constant_(m.bias, 0)


def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True


def call(data,name,num_features,num_classes):
    val_prop = 0.05
    test_prop = 0.1
    train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false = get_edges_split(data, val_prop = val_prop, test_prop = test_prop)
    total_edges = np.concatenate((train_edges,train_edges_false,val_edges,val_edges_false,test_edges,test_edges_false))
    data.train_pos,data.train_neg = len(train_edges),len(train_edges_false)
    data.val_pos, data.val_neg = len(val_edges), len(val_edges_false)
    data.test_pos, data.test_neg = len(test_edges), len(test_edges_false)
    data.total_edges = total_edges
    data.total_edges_y = torch.cat((torch.ones(len(train_edges)), torch.zeros(len(train_edges_false)), torch.ones(len(val_edges)), torch.zeros(len(val_edges_false)),torch.ones(len(test_edges)), torch.zeros(len(test_edges_false)))).long()

    # delete val_pos and test_pos
    edge_list = np.array(data.edge_index).T.tolist()
    for edges in val_edges:
        edges = edges.tolist()
        if edges in edge_list:
            # if not in edge_list, mean it is a self loop
            edge_list.remove(edges)
            edge_list.remove([edges[1], edges[0]])
    for edges in test_edges:
        edges = edges.tolist()
        if edges in edge_list:
            edge_list.remove(edges)
            edge_list.remove([edges[1], edges[0]])
    data.edge_index = torch.Tensor(edge_list).long().transpose(0, 1)

    # edge index sampling
    random_edge_num = 2500
    indices = np.random.choice((data.edge_index).size(1), (random_edge_num,), replace=False)
    indices = np.sort(indices)
    sample_data_edge_index = data.edge_index[:, indices]

    boundary_matrix0_, boundary_matrix1_ = compute_boundary_matrix(data, sample_data_edge_index)

    L0u, L1f = compute_hodge_matrices(boundary_matrix0_, boundary_matrix1_)
    L0u = torch.tensor(L0u, dtype=torch.float32)  # convert hodge matrix to tensor format
    L1f = torch.tensor(L1f, dtype=torch.float32)
    #print(L0u.size())
    #print(L1f.size())
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data.total_edges_y.to(device)
    model, data = SIMGAE(data, num_features, num_classes,
                               hodge_laplacians=[L0u.to(device), L1f.to(device)]).to(device), data.to(device)
    return model, data

### 6. Now, use SIMGAE to run the experiments of Link prediction

In [6]:
dataset=Planetoid('./data/'+"Cora","Cora",transform=T.NormalizeFeatures())
# dataset = PygLinkPropPredDataset(name="ogbl-ddi", root='./dataset/') #download the dataset

wait_total= 300
total_epochs = 4000


#print("test")
data = dataset[0]
model, data = call(data, dataset.name, data.x.size(1), dataset.num_classes)
model.apply(weights_init)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)
best_val_acc = test_acc_same = test_acc_diff = test_acc = 0.0
best_val_roc = test_roc_same = test_roc_diff = test_roc = 0.0
best_val_loss = np.inf
# train and val/test
# train and val/test
wait_step = 0
for epoch in range(1, total_epochs + 1):
    
    pred = train()
    val_loss, val_roc, val_acc, tmp_test_roc, tmp_test_acc = test()
    if val_roc >= best_val_roc:
        test_acc = tmp_test_acc
        test_roc = tmp_test_roc
        best_val_acc = val_acc
        best_val_roc = val_roc
        best_val_loss = val_loss
        wait_step = 0
    else:
        wait_step += 1
        if wait_step == wait_total:
            print('Early stop! Min loss: ', best_val_loss, ', Max accuracy: ', best_val_acc,', Max roc: ', best_val_roc)
            break
    if epoch %20==0:
        print("epoch is:", epoch)
        print(best_val_roc)


(2708, 2154)
(2154, 131)
epoch is: 20
0.8006332316500167
epoch is: 40
0.8646648064884558
epoch is: 60
0.8761294799693504
epoch is: 80
0.9027888215819224
epoch is: 100
0.9224074368575518
epoch is: 120
0.9368503231216297
epoch is: 140
0.9429802368112883
epoch is: 160
0.9442091110179416
epoch is: 180
0.9442091110179416
epoch is: 200
0.946262053810233
epoch is: 220
0.9495872428400006
epoch is: 240
0.9496306148708237
epoch is: 260
0.9496306148708237
epoch is: 280
0.9500065058046234
epoch is: 300
0.951640185632292
epoch is: 320
0.9534762682704678
epoch is: 340
0.9534762682704678
epoch is: 360
0.9534762682704678
epoch is: 380
0.9534762682704678
epoch is: 400
0.9534762682704678
epoch is: 420
0.9534762682704678
epoch is: 440
0.9534762682704678
epoch is: 460
0.9534762682704678
epoch is: 480
0.9551533201289595
epoch is: 500
0.9571195188596047
epoch is: 520
0.9571195188596047
epoch is: 540
0.9571195188596047
epoch is: 560
0.9571195188596047
epoch is: 580
0.9571195188596047
epoch is: 600
0.95711951